In [1]:
import sys
from tqdm import tqdm
import pdb
import torch
import logging
import numpy as np
import dgl
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split
import random

In [2]:
train_path = 'train.txt'
val_path = 'val.txt'
test_path = 'test.txt'
category_path = 'item_category.txt'

In [4]:
historical_dict = {}
cat_dict={}
dic = {}
with open(category_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        item = int(line[0])
        category = int(line[1])            
        dic[item] = category
with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]
           cat_dict[user]=[dic[item]]
with open(val_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]
           cat_dict[user]=[dic[item]]
with open(test_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]     
           cat_dict[user]=[dic[item]]    

100%|██████████| 16399/16399 [00:00<00:00, 1203288.75it/s]


In [6]:
cat_dict

{1: [7, 9, 6, 7, 12, 7, 7, 7, 9, 1, 17, 7, 12, 1, 5, 7, 9, 7, 7, 7],
 2: [6, 20, 17, 7, 7, 20, 1, 36, 14],
 3: [5, 14, 14, 7, 1, 6, 5, 5, 14],
 4: [7, 5, 1, 7, 17],
 5: [7, 7, 38, 1, 1, 9, 6, 14],
 6: [31, 9, 1, 8, 1, 9, 2, 21, 10, 7, 9],
 7: [7, 1, 6, 23, 5, 13, 7, 13, 2, 5, 13, 6],
 8: [5, 8, 7, 1, 2],
 9: [9, 9, 7, 7, 9, 2, 12, 9],
 10: [23,
  2,
  8,
  2,
  20,
  7,
  5,
  2,
  18,
  7,
  7,
  3,
  9,
  7,
  7,
  6,
  6,
  7,
  7,
  7,
  12,
  1,
  7,
  7,
  23,
  7,
  6,
  31,
  7,
  7,
  16,
  7,
  4,
  6,
  7,
  15,
  7,
  6,
  7,
  7,
  14,
  2,
  7,
  23,
  31,
  5,
  23,
  21],
 11: [7, 5, 2, 12, 12],
 12: [9, 7, 7, 2, 14, 17, 15, 18],
 13: [7, 6, 2, 38, 14, 14, 7, 7, 14, 14, 7, 7, 6, 15, 7, 14, 9, 7, 6, 23, 7],
 14: [14, 12, 25, 14, 22, 2],
 15: [8, 2, 23, 23, 23, 18],
 16: [7, 13, 7, 14, 9, 7, 22, 2, 9, 9, 9],
 17: [7, 7, 5, 9, 2, 7, 5],
 18: [7, 2, 7, 7, 23, 39, 7],
 19: [3, 17, 2, 5, 7, 12, 7],
 20: [2,
  2,
  18,
  8,
  2,
  2,
  1,
  7,
  16,
  32,
  17,
  7,
  7,
  2,


In [7]:
train={}
test={}
val={}

for i in historical_dict.keys():
    train[i]=[-1]
    test[i]=[-1]
    val[i]=[-1]
    unique_elements = list(set(cat_dict[i]))
    element_counts = [cat_dict[i].count(element) for element in unique_elements]
    element_indices = {}
    for index, element in enumerate(cat_dict[i]):
        if element in element_indices:
            element_indices[element].append(index)
        else:
            element_indices[element] = [index]
    
    for j in unique_elements:
        my_list=element_indices[j]
        if(len(my_list)<=3):
            for k in my_list:
                rand=random.random()
                if rand<0.6:
                    train[i].append(historical_dict[i][k])
                elif rand>0.6 and rand<0.8:
                    test[i].append(historical_dict[i][k])
                else:
                    val[i].append(historical_dict[i][k])
        else:
            result_list = [historical_dict[i][index] for index in my_list]
            train_sp,temp_sp,_,_=train_test_split(result_list, result_list, test_size=0.4, random_state=42)
            train[i]+=train_sp
            test_sp,val_sp,_,_=train_test_split(temp_sp, temp_sp, test_size=0.5, random_state=42)
            test[i]+=test_sp
            val[i]+=val_sp

In [8]:
print(historical_dict[i])
print(cat_dict[i])
print(unique_elements)
print(element_counts)
print(element_indices)
print(train[i])
print(test[i])
print(val[i])

[223]
[13]
[13]
[1]
{13: [0]}
[-1, 223]
[-1]
[-1]


In [9]:
file_path = "train_v2.txt"
train_num=0
test_num=0
val_num=0
with open(file_path, "w") as file:
    for user in train.keys():
        for item in train[user]:
            if item==-1:
                continue
            else:
                train_num+=1
                file.write(str(user)+','+str(item)+"\n")
file_path = "test_v2.txt"
with open(file_path, "w") as file:
    for user in test.keys():
        for item in test[user]:
            if item==-1:
                continue
            else:
                test_num+=1
                file.write(str(user)+','+str(item)+"\n")
file_path = "val_v2.txt"
with open(file_path, "w") as file:
    for user in val.keys():
        for item in val[user]:
            if item==-1:
                continue
            else:
                val_num+=1
                file.write(str(user)+','+str(item)+"\n")
        